## Batch-3 Aug 

## Group-9  Avi Koyani

In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("cap_stone").getOrCreate()
import pyspark.sql.functions as psf

## Imporing Data

In [3]:
Txn = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('Car_sales_transactions.csv')

In [4]:
Txn.printSchema()

root
 |-- Sales_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Selling_Price: integer (nullable = true)
 |-- km_Driven: integer (nullable = true)
 |-- City_Code: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Postal_Code: integer (nullable = true)
 |-- Fuel: string (nullable = true)
 |-- Seller_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Engine: string (nullable = true)
 |-- Max_Power: string (nullable = true)
 |-- Seats: integer (nullable = true)



In [5]:
City_Master = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('City_Master.csv')
Postal_Code_Master = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('Postal_Code_Master.csv')
Region_Master = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('Region_Master.csv')
Region_State_Mapping = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('Region_State_Mapping.csv')
Sales_Status = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('Sales_Status.csv')
State_Master = spark.read.options(inferschema = 'True',delimiter = ',',header = 'True')\
      .csv('State_Master.csv')

In [6]:
Postal_Code_Master.rdd.getNumPartitions()

1

In [7]:
Region_Master.printSchema()

root
 |-- Region_Code: string (nullable = true)
 |-- Region: string (nullable = true)



## Joining data

In [8]:
master_pyspark = Txn.join(City_Master,"City_Code","left").join(Region_State_Mapping,"State_Code","left")\
   .join(Sales_Status,"Sales_ID","left").join(State_Master,"State_Code","left")\
   .join(Region_Master,"Region_Code","left")

In [9]:
master_pyspark.printSchema()

root
 |-- Region_Code: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Sales_ID: integer (nullable = true)
 |-- City_Code: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Selling_Price: integer (nullable = true)
 |-- km_Driven: integer (nullable = true)
 |-- Postal_Code: integer (nullable = true)
 |-- Fuel: string (nullable = true)
 |-- Seller_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Engine: string (nullable = true)
 |-- Max_Power: string (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- City_Name: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Sold: string (nullable = true)
 |-- State_Name: string (nullable = true)
 |-- Region: string (nullable = true)



## Extracting Imp Information From columns

In [10]:
Split_Mileage = pyspark.sql.functions.split(master_pyspark['Mileage']," ")
#Split_Power = master_pyspark.select(split("Max_Power"," ").as("a")).drop("Max_Power")
Split_Power = psf.split(master_pyspark["Max_Power"]," ")
Split_Engine = psf.split(master_pyspark["Engine"]," ")
Split_Name = psf.split(master_pyspark['Name']," ")

In [11]:
Split_Power

Column<'split(Max_Power,  , -1)'>

In [12]:
Split_Mileage

Column<'split(Mileage,  , -1)'>

In [13]:
master_pyspark1 = master_pyspark.withColumn('Mileage',Split_Mileage.getItem(0).cast("integer"))\
        .withColumn('Max_Power',Split_Power.getItem(0).cast("integer"))\
        .withColumn('Engine',Split_Engine.getItem(0).cast("integer"))\
        .withColumn('Brand',Split_Name.getItem(0))\
        .withColumn('Model',Split_Name.getItem(1))

## Dropping Unnecessary columns

In [14]:
master_pyspark2 = master_pyspark1.drop('Region_Code').drop('State_Code').drop('City_Code').drop('Name')

In [15]:
master_pyspark2.printSchema()

root
 |-- Sales_ID: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Selling_Price: integer (nullable = true)
 |-- km_Driven: integer (nullable = true)
 |-- Postal_Code: integer (nullable = true)
 |-- Fuel: string (nullable = true)
 |-- Seller_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Mileage: integer (nullable = true)
 |-- Engine: integer (nullable = true)
 |-- Max_Power: integer (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- City_Name: string (nullable = true)
 |-- Sold: string (nullable = true)
 |-- State_Name: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)



In [16]:
master_pyspark2.select('Brand').show(5)

+-------+
|  Brand|
+-------+
| Maruti|
|  Skoda|
|  Honda|
|Hyundai|
| Maruti|
+-------+
only showing top 5 rows



In [17]:
master_pyspark2 = master_pyspark2.withColumn("Brand", psf.when(master_pyspark2.Brand == "Land","Land Rover")\
                                             .otherwise(master_pyspark2.Brand))

In [18]:
master_pyspark2.select("Brand").where(master_pyspark2.Brand == "Land Rover").show()

+----------+
|     Brand|
+----------+
|Land Rover|
|Land Rover|
|Land Rover|
|Land Rover|
|Land Rover|
|Land Rover|
+----------+



## Segregating Sold and unsold cars

In [19]:
sold_pyspark = master_pyspark2.filter(master_pyspark2['Sold'] == 'Y')
unsold_pyspark = master_pyspark2.filter(master_pyspark2['Sold'] == 'N')

In [20]:
sold_pyspark.count()

2000

In [21]:
unsold_pyspark.count()

5906

## Q3 Brand Wise sales

In [22]:
Brand_Wise = sold_pyspark.groupby("Brand").agg(psf.count("Sales_ID").alias("Count"),psf.avg("Selling_Price").alias("Avg_Selling_Price"))\
     .sort(psf.desc("Count"))

In [23]:
Brand_Wise.show()

+-------------+-----+------------------+
|        Brand|Count| Avg_Selling_Price|
+-------------+-----+------------------+
|       Maruti|  597| 405231.9631490787|
|      Hyundai|  332|          463259.0|
|     Mahindra|  201| 558303.4278606966|
|         Tata|  178|  369480.297752809|
|       Toyota|  126| 1085555.507936508|
|        Honda|  124| 582581.8225806452|
|         Ford|   92| 562228.2282608695|
|      Renault|   56|419035.66071428574|
|    Chevrolet|   56|279499.96428571426|
|   Volkswagen|   39| 492487.1538461539|
|       Nissan|   33|420969.63636363635|
|          BMW|   29| 4606551.724137931|
|       Jaguar|   21|2917857.1428571427|
|        Skoda|   20|          747749.9|
|        Volvo|   19|3547368.4210526315|
|Mercedes-Benz|   16|         2767187.5|
|       Datsun|   16|       297249.9375|
|        Lexus|   11|         5150000.0|
|         Jeep|    9|2551666.5555555555|
|         Audi|    9|2637777.6666666665|
+-------------+-----+------------------+
only showing top

## Q4 Loaction wise

In [24]:
Location_Wise = sold_pyspark.groupby("Region","State_Name").agg(psf.count("Sales_ID").alias("Count")).sort(psf.desc("Count"))

In [25]:
Location_Wise.show()

+----------+--------------+-----+
|    Region|    State_Name|Count|
+----------+--------------+-----+
|     South|    Tamil Nadu|  230|
|     South|     Karnataka|  223|
|      West|       Gujarat|  208|
|      West|   Maharashtra|  189|
|   Central| Uttar Pradesh|  145|
|     South|        Kerala|  130|
|      East|     Jharkhand|  127|
|     North|         Delhi|   80|
|     South|     Telangana|   72|
|   Central|   Uttarakhand|   70|
|   Central|Madhya Pradesh|   69|
|      East|   West Bengal|   69|
|North East|        Sikkim|   69|
|      East|         Bihar|   69|
|     South|Andhra Pradesh|   68|
|      East|        Odisha|   64|
|     North|        Punjab|   63|
|     North|     Rajasthan|   55|
+----------+--------------+-----+



### Revenue Generated:

##### Brandwise

In [26]:
brand_revenue = sold_pyspark.groupBy("Brand").sum("Selling_Price")
brand_revenue = brand_revenue.select("Brand", psf.col("sum(Selling_Price)").alias("Revenue"))

from pyspark.sql.functions import col
brand_revenue.sort(col("Revenue").desc()).show(10)

+--------+---------+
|   Brand|  Revenue|
+--------+---------+
|  Maruti|241923482|
| Hyundai|153801988|
|  Toyota|136779994|
|     BMW|133590000|
|Mahindra|112218989|
|   Honda| 72240146|
|   Volvo| 67400000|
|    Tata| 65767493|
|  Jaguar| 61275000|
|   Lexus| 56650000|
+--------+---------+
only showing top 10 rows



#### Statewise:


In [27]:
state_revenue = sold_pyspark.groupBy("State_Name").sum("Selling_Price")
state_revenue = state_revenue.select("State_Name", col("sum(Selling_Price)").alias("Revenue"))

from pyspark.sql.functions import col
state_revenue.sort(col("Revenue").desc()).show(10)

+--------------+---------+
|    State_Name|  Revenue|
+--------------+---------+
|    Tamil Nadu|151438988|
|     Karnataka|143235148|
|       Gujarat|141656988|
|   Maharashtra|110386992|
|     Jharkhand| 95757995|
| Uttar Pradesh| 91596500|
|        Kerala| 86655995|
|Andhra Pradesh| 74728995|
|         Delhi| 59100996|
|   West Bengal| 52936996|
+--------------+---------+
only showing top 10 rows



#### Top 10 States & Brands Revenue wise:  

In [28]:
from pyspark.sql.functions import col
statebrand_revenue = sold_pyspark.groupBy("State_Name","Brand").sum("Selling_Price")
statebrand_revenue= statebrand_revenue.select("State_Name","Brand", col("sum(Selling_Price)").alias("Revenue")).sort(col("Revenue").desc()).show(10)


+-----------+--------+--------+
| State_Name|   Brand| Revenue|
+-----------+--------+--------+
|  Karnataka|  Maruti|29020000|
|    Gujarat|  Maruti|27308996|
| Tamil Nadu|  Maruti|26353997|
|  Karnataka| Hyundai|20889999|
|Maharashtra|  Maruti|20516996|
|  Jharkhand|     BMW|20250000|
|     Kerala|  Maruti|18832000|
|     Kerala|  Toyota|18235998|
| Tamil Nadu|Mahindra|18182999|
| Tamil Nadu| Hyundai|17312000|
+-----------+--------+--------+
only showing top 10 rows



#### Yearwise sales :

In [29]:
from pyspark.sql.functions import sum,avg,max,count,min
year_sales=sold_pyspark.groupBy("Year").agg(count("*").alias("Sales Number")).sort(col("Sales Number").desc()).show(25)

+----+------------+
|Year|Sales Number|
+----+------------+
|2017|         256|
|2016|         225|
|2015|         200|
|2018|         187|
|2013|         174|
|2012|         159|
|2011|         155|
|2019|         154|
|2014|         144|
|2010|          96|
|2009|          71|
|2008|          43|
|2007|          28|
|2005|          25|
|2006|          23|
|2020|          19|
|2004|          13|
|2003|          10|
|2002|           7|
|2000|           5|
|1999|           3|
|2001|           2|
|1994|           1|
+----+------------+



#### Most selling Brands Yearwise:

In [30]:
year_sales=sold_pyspark.groupBy("Year","Brand").agg(count("*").alias("Unit_Sold")).sort(col("Unit_Sold").desc()).show(10)

+----+-------+---------+
|Year|  Brand|Unit_Sold|
+----+-------+---------+
|2017| Maruti|       73|
|2016| Maruti|       65|
|2015| Maruti|       64|
|2018| Maruti|       60|
|2017|Hyundai|       50|
|2012| Maruti|       47|
|2014| Maruti|       46|
|2013| Maruti|       45|
|2015|Hyundai|       42|
|2019| Maruti|       36|
+----+-------+---------+
only showing top 10 rows



#### Most selling Units based on CC

In [31]:
year_sales=sold_pyspark.groupBy("Engine").agg(count("*").alias("Sales Number")).sort(col("Sales Number").desc()).show(10)

+------+------------+
|Engine|Sales Number|
+------+------------+
|  1248|         239|
|  1197|         214|
|   998|         116|
|   796|         115|
|  2179|          94|
|  1498|          81|
|  1396|          80|
|  1199|          68|
|  2494|          61|
|  2523|          58|
+------+------------+
only showing top 10 rows



### Selling Price Analysis:

#### Yearwise Analysis:

In [32]:

yearwise_sellingprice=sold_pyspark.groupBy("Year").agg(  sum("Selling_Price").alias("Total_Selling_Price"), avg("Selling_Price").alias("avg_Selling_Price"),  
                      max("Selling_Price").alias("max_Selling_Price") , min("Selling_Price").alias("min_Selling_Price")) .sort(col("Total_Selling_Price").desc())


In [33]:
from pyspark.sql.functions import round
yearwise_sellingprice.select("*", round(col('avg_Selling_Price'),3)).show(25)

+----+-------------------+------------------+-----------------+-----------------+---------------------------+
|Year|Total_Selling_Price| avg_Selling_Price|max_Selling_Price|min_Selling_Price|round(avg_Selling_Price, 3)|
+----+-------------------+------------------+-----------------+-----------------+---------------------------+
|2019|          283122996| 1838461.012987013|          5800000|           270000|                1838461.013|
|2017|          253754990|    991230.4296875|         10000000|           200000|                  991230.43|
|2018|          203527996|1088385.0053475937|          6000000|           217000|                1088385.005|
|2016|          157261995|          698942.2|          3000000|           150000|                   698942.2|
|2015|          109182991|        545914.955|          2650000|           200000|                 545914.955|
|2013|           87153989| 500884.9942528736|          3750000|            80000|                 500884.994|
|2014|    

#### Brandwise Analysis:

In [34]:

brandwise_sellingprice=sold_pyspark.groupBy("Brand").agg( count("*").alias("Units_Sold"), sum("Selling_Price").alias("Total_Selling_Price"), round (avg("Selling_Price"),2).alias("avg_Selling_Price"),  
                      max("Selling_Price").alias("max_Selling_Price") , min("Selling_Price").alias("min_Selling_Price")).sort(col("avg_Selling_Price").desc()).show(10)

#brandwise_sellingprice.select("*", round(col('avg_Selling_Price'),3)).show(10)


+-------------+----------+-------------------+-----------------+-----------------+-----------------+
|        Brand|Units_Sold|Total_Selling_Price|avg_Selling_Price|max_Selling_Price|min_Selling_Price|
+-------------+----------+-------------------+-----------------+-----------------+-----------------+
|        Lexus|        11|           56650000|        5150000.0|          5150000|          5150000|
|          BMW|        29|          133590000|       4606551.72|          6000000|           975000|
|   Land Rover|         1|            4500000|        4500000.0|          4500000|          4500000|
|        Volvo|        19|           67400000|       3547368.42|         10000000|          2475000|
|       Jaguar|        21|           61275000|       2917857.14|          3200000|          2450000|
|Mercedes-Benz|        16|           44275000|        2767187.5|          6000000|           800000|
|         Audi|         9|           23739999|       2637777.67|          3975000|         

## Encoding

In [35]:
usedCarDF = usedCarDF = spark.read.load("result_final.csv", format="csv", sep=",", inferSchema="true", header="true")

In [36]:
Split_Power = psf.split(usedCarDF["Max_Power"]," ")

In [37]:
usedCarDF = usedCarDF.withColumn("Max_Power",Split_Power.getItem(0).cast("integer"))

In [38]:
import pyspark.sql.functions as F

In [39]:
fuel = usedCarDF.select("FUEL").distinct().rdd.flatMap(lambda x: x).collect()
transmission = usedCarDF.select("TRANSMISSION").distinct().rdd.flatMap(lambda x: x).collect()
fuelType = [F.when(F.col("FUEL") == ty, 1).otherwise(0).alias("" + ty) for ty in fuel]
transmissionType = [F.when(F.col("TRANSMISSION") == code, 1).otherwise(0).alias("" + code) for code in transmission]
usedCarDF = usedCarDF.select("SELLING_PRICE", "YEAR", "KM_DRIVEN","MAX_POWER","SEATS","MILEAGE_NO","SOLD",*fuelType+transmissionType)
usedCarDF_Sold = usedCarDF.filter(usedCarDF.SOLD=="Y").drop("SOLD")
usedCarDF_unSold=usedCarDF.filter(usedCarDF.SOLD=="N").drop("SOLD")
usedCarDF_Sold.show()

+-------------+----+---------+---------+-----+----------+------+---+---+------+---------+------+
|SELLING_PRICE|YEAR|KM_DRIVEN|MAX_POWER|SEATS|MILEAGE_NO|Diesel|CNG|LPG|Petrol|Automatic|Manual|
+-------------+----+---------+---------+-----+----------+------+---+---+------+---------+------+
|       450000|2014|   145500|       74|    5|      23.4|     1|  0|  0|     0|        0|     1|
|       370000|2014|   120000|      103|    5|     21.14|     1|  0|  0|     0|        0|     1|
|       158000|2006|   140000|       78|    5|      17.7|     0|  0|  0|     1|        0|     1|
|       225000|2010|   127000|       90|    5|      23.0|     1|  0|  0|     0|        0|     1|
|       130000|2007|   120000|       88|    5|      16.1|     0|  0|  0|     1|        0|     1|
|       440000|2017|    45000|       81|    5|     20.14|     0|  0|  0|     1|        0|     1|
|        96000|2007|   175000|       57|    5|      17.3|     0|  0|  1|     0|        0|     1|
|        45000|2001|     5000|

## Finding Correlation of different features with selling price

In [40]:
column_list= usedCarDF_Sold.columns

## Correlation for Sold Cars

In [41]:
for i in column_list:
    print(f'{i:15}   {usedCarDF_Sold.corr("SELLING_PRICE",i)}')

SELLING_PRICE     1.0
YEAR              0.41633780251166896
KM_DRIVEN         -0.23536376181512805
MAX_POWER         0.7787732815949355
SEATS             0.009365095128508102
MILEAGE_NO        -0.10077194169192169
Diesel            0.19098213504877337
CNG               -0.021031438766946558
LPG               -0.02943889643545322
Petrol            -0.18541170589683892
Automatic         0.6030166122876204
Manual            -0.6030166122876205


## Correlation for unsold Cars

In [64]:
for i in column_list:
    print(f'{i:15}   {usedCarDF_unSold.corr("SELLING_PRICE",i)}')

SELLING_PRICE     1.0
YEAR              0.41195436159198523
KM_DRIVEN         -0.21767236571895754
MAX_POWER         0.7389628647884097
SEATS             0.05384188847352593
MILEAGE_NO        -0.1355470382306577
Diesel            0.21114703476810504
CNG               -0.0366968418394256
LPG               -0.0381722432806315
Petrol            -0.19993798762929735
Automatic         0.5853797441685367
Manual            -0.5853797441685367
